In [1]:
import datetime
import json
import pandas as pd

In [2]:
json_path = 'data/attestations.json'
with open(json_path, 'r') as f:
    data = json.load(f)
print(len(data))    

149


In [3]:
relevant_attestations = []
for a in data:
    qs = a.get('metadata', {}).get('impactAttestations', [])
    
    if len(qs) < 2:
        continue
    
    nps = pmf = None
    for q in qs:
        val = q.get('value')
        if not val:
            continue
        if q['name'] == 'Likely to Recommend':
            nps = val
        elif q['name'] == 'Feeling if didnt exist':
            pmf = val
    
    if not (nps or pmf):
        continue
        
    relevant_attestations.append({
        'id': a['id'],
        'attester': a['attester'],
        'recipient': a['recipient'],
        'farcasterID': a['farcasterID'],
        'projectRegUID': a['projectRegUID'],
        'timeCreated': datetime.datetime.utcfromtimestamp(a['timeCreated']),
        'issuer': a['issuer'],
        'metadataurl': a['metadataurl'],
        'nps_score': pd.to_numeric(nps),
        'pmf_score': pd.to_numeric(pmf)
    })

In [4]:
df_attestations_actual = pd.DataFrame(relevant_attestations)
df_attestations_actual.tail(1)

,id,attester,recipient,farcasterID,projectRegUID,timeCreated,issuer,metadataurl,nps_score,pmf_score
15,0xecc8d17aa547682ffc8b4b69c1ed59bd642fad4dc6c8...,0x7484aABFef9f39464F332e632047983b67571C0a,0x5d36a202687fD6Bd0f670545334bF0B4827Cc1E2,"{'type': 'BigNumber', 'hex': '0x2d4c'}",0x82169691e4026be1d950f20c2d36500b06fb6f3e0c03...,2024-10-02 06:16:34,MGL,https://gateway.pinata.cloud/ipfs/QmeYyTJzXsXS...,6,2.0


# Create some dummy data

1. Number of Attestations (only want the count from Citizens and Top Delegates)
2. Number of Attestations by Citizens
3. Number of Attestations by Top delegates
4. Average NPS score of Citizens and Top delegates
5. Most positive superlative
6. "Can't live without" superlative
7. Percentage distribution of different ratings by citizens and top delegates [Extremely upset, Somewhat upset, Neutral]
    1. Percentage distribution of different ratings by citizens
    2. Percentage distribution of different ratings by delegates
8. Number of elected governance members that attested
    1. Number of each group of elected governance members that attested
    2. Average NPS score per elected governance group
    3. Average feeling per elected governance group

In [5]:
import numpy as np
import random
import secrets

In [6]:
NUM_PROJECTS = 30
NUM_ATTESTERS = 100
NUM_ATTESTATIONS = 1000
PMF_SCALE = 5
NPS_SCALE = 10

projects = {
    f"0x{secrets.token_hex(32)}": random.random()
    for _ in range(NUM_PROJECTS)
}

attesters = {}
for a in range(NUM_ATTESTERS):
    addr = f"0x{secrets.token_hex(20)}"
    citizen = random.randint(1,10)
    delegate = random.randint(1,10)    
    attesters.update({
        addr:{
            'is_citizen': citizen > 3,
            'is_top_delgate': delegate > 6
        }
    })

attestations = []
for i in range(NUM_ATTESTATIONS):
    random_project = random.choice(list(projects.keys()))
    random_attester = random.choice(list(attesters.keys()))
    project_impact = projects.get(random_project)
    pmf_score = random.randint(int(project_impact*PMF_SCALE), PMF_SCALE)
    nps_score = random.randint(int(project_impact*NPS_SCALE), NPS_SCALE)
    attestations.append({
        'uid': f"0x{secrets.token_hex(32)}",
        'attester': random_attester,
        'projectRegUID': random_project,
        'pmf_score': max(1,pmf_score),
        'nps_score': max(1,nps_score),
        **attesters[random_attester]
    })

In [7]:
df_sim = pd.DataFrame(attestations)
df_sim.tail(1)

,uid,attester,projectRegUID,pmf_score,nps_score,is_citizen,is_top_delgate
999,0xf62ceb87e3a6f80a98a4e8c2efcdfc186b82a183e5d0...,0xe8423d0d55ad3db2d2e12c8f60bb1c7bcffb6c77,0xefbc22cf4df5bc0f15968061c459fa6c876d7741b51e...,4,1,True,False


In [8]:
count_total_attestations = df_sim.groupby('projectRegUID')['uid'].count().rename('count_total_attestations')
count_citizen_attestations = df_sim[df_sim.is_citizen].groupby('projectRegUID')['uid'].count().rename('count_citizen_attestations')
count_delegate_attestations = df_sim[df_sim.is_top_delgate].groupby('projectRegUID')['uid'].count().rename('count_delegate_attestations')
avg_nps_score = df_sim.groupby('projectRegUID')['nps_score'].mean().rename('avg_nps_score')
avg_nps_score

projectRegUID
0x1d93dc2c0b61d437e2c4beb9d806e93471664cd3b45f4674f0d8fe008afafc27    8.277778
0x27f547a11e4ddfaa888c181b239dd4c996370c95485d988e927bc20a69a05920    8.181818
0x2c0469ba04de85641373e91a04644a0f0042180ce6381efc668f2a7f898acb9b    4.481481
0x3197addb2d3f675ac6976ad83ee87b6cd70cd56e04bd6dab0e4368425aed0265    9.444444
0x3e0f7e5493a325aab054a861d5b161238d45a5072bef9a21e9244dd078567831    8.269231
0x3f57c8deb5d2304490062e5ee01fda1dc0b9d100684db724d5e08ba6671a7c7d    8.944444
0x548c5383763131e05dae98c800ffd32f23dce2ace5607fb16f23d7c62d613e72    7.862069
0x5d064203a908677a8766814c6f77f6891d8363d9ea43001b8f0ac21c6362a25e    6.000000
0x6308c9af99058f2947dc4c1cb1c0909f81718948274961b9b9c964f7dadca0b2    6.850000
0x67cf166f33b0bd79434eca44914e2751b38b898fb579d0388fc5aa38c6ecd97a    8.357143
0x6baba63ac23d75d296e3ffab653fde6943b1dae9c3e8cdb57c7315e2dc2a51fb    8.216216
0x70faa8a471a1d5be20f7ad453b7203b1d671bc8dfbf641a5b1cde7bb7576cdd9    7.238095
0x712529d38aaeabc03a484326a42ec4b33c78